In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_regression').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [9]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

In [10]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [15]:
for d in data.head(1)[0]:
    print(d)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [18]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [19]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [20]:
assembler = VectorAssembler(
    inputCols=['Avg Session Length',
                 'Time on App',
                 'Time on Website',
                 'Length of Membership'],
    outputCol='features')

In [21]:
output = assembler.transform(data)

In [22]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [23]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [24]:
final_data = output.select(['features','Yearly Amount Spent'])

In [25]:
final_data.show(5)

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
+--------------------+-------------------+
only showing top 5 rows



In [26]:
train_data, test_data = final_data.randomSplit([.7, .3],seed=42)

In [30]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                374|
|   mean| 499.78515710377627|
| stddev|  79.76963867632105|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [31]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                126|
|   mean|  497.9156378768501|
| stddev|  78.24705567534872|
|    min|  282.4712457199145|
|    max|  744.2218671047146|
+-------+-------------------+



In [33]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [34]:
lr_model = lr.fit(train_data)

In [36]:
lr_model

LinearRegressionModel: uid=LinearRegression_b8f10edbe2a6, numFeatures=4

In [38]:
test_result = lr_model.evaluate(test_data)

In [41]:
test_result.residuals.show(5)

+-------------------+
|          residuals|
+-------------------+
| -3.991172060389374|
|-3.5156955606657334|
|  7.407581473990376|
|  4.754112229314956|
| -7.287952052180344|
+-------------------+
only showing top 5 rows



In [43]:
test_result.r2adj

0.9843048045580759

In [44]:
test_result.rootMeanSquaredError

9.60635830573073

In [45]:
test_result.r2

0.9848070508122174

In [52]:
unlabeled_data = test_data.select('features')

In [54]:
unlabeled_data.show(3)

+--------------------+
|            features|
+--------------------+
|[30.4925366965402...|
|[30.8364326747734...|
|[30.9716756438877...|
+--------------------+
only showing top 3 rows



In [56]:
predition = lr_model.transform(unlabeled_data)

In [57]:
predition.show(3)

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...| 286.4624177803039|
|[30.8364326747734...|471.01759598765534|
|[30.9716756438877...|487.23102828290234|
+--------------------+------------------+
only showing top 3 rows



In [ ]:
predition.ra